In [ ]:
import os
from langchain.document_loaders import TextLoader

root_dir = "./path/to/cloned/repository"

docs = []
file_extensions = []

for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        file_path = os.path.join(dirpath, file)
        if file_extensions and os.path.splitext(file)[1] not in  file_extensions:
            continue

    loader = TextLoader(file_path, encoding = 'utf-8')
    docs.extend(loader.load_and_split())


from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
splitted_text = text_splitter.split_documents(docs)

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

embeddings = OpenAIEmbeddings(model = 'text-embedding-ada-002')

user = "veaceslavcalestru"
dataset = 'langchain_course_chat_with_gh'
dataset_path = f"hub://{user}/{dataset}"

db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)
db.add_documents(splitted_text)

retriever = db.as_retriever()

retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['fetch_k'] = 100
retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 10

model = ChatOpenAI()

qa = RetrievalQA.from_llm(model, retriever=retriever)

qa.run("What is the repository's name?")